In [49]:
import sys
import os
from collections import OrderedDict

from ray_tools.base.engine import MinMaxRayEngine, RayEngine
from ray_tools.base.backend import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
file_root = ''

param_limit_dict = OrderedDict([
    #('U41_318eV.numberRays', 100),

    ('U41_318eV.translationXerror', (-0.25, 0.25)),
    ('U41_318eV.translationYerror', (-0.25, 0.25)),
    ('U41_318eV.rotationXerror', (-0.05, 0.05)),
    ('U41_318eV.rotationYerror', (-0.05, 0.05)),

    ('ASBL.openingWidth', (1.9, 2.1)),
    ('ASBL.openingHeight', (0.9, 1.1)),
    ('ASBL.translationXerror', (-0.2, 0.2)),
    ('ASBL.translationYerror', (-0.2, 0.2)),

    ('M1_Cylinder.radius', (174.06, 174.36)),
    ('M1_Cylinder.rotationXerror', (-0.25, 0.25)),
    ('M1_Cylinder.rotationYerror', (-1.0, 1.0)),
    ('M1_Cylinder.rotationZerror', (-1.0, 1.0)),
    ('M1_Cylinder.translationXerror', (-1.0, 1.0)),
    ('M1_Cylinder.translationYerror', (-1.0, 1.0)),

    ('SphericalGrating.radius', (109741.0, 109841.0)),
    ('SphericalGrating.rotationYerror', (-1.0, 1.0)),
    ('SphericalGrating.rotationZerror', (-2.5, 2.5)),

    ('ExitSlit.openingHeight', (0.009, 0.011)),
    ('ExitSlit.translationZerror', (-150.0, 150.0)),
    ('ExitSlit.rotationZerror', (-0.3, 0.3)),

    ('E1.longHalfAxisA', (20600.0, 20900.0)),
    ('E1.shortHalfAxisB', (300.721702601, 304.721702601)),
    ('E1.rotationXerror', (-1.5, 1.5)),
    ('E1.rotationYerror', (-7.5, 7.5)),
    ('E1.rotationZerror', (7.0, 14.0)),
    ('E1.translationYerror', (-1.0, 1.0)),
    ('E1.translationZerror', (-1.0, 1.0)),

    ('E2.longHalfAxisA', (4325.0, 4425.0)),
    ('E2.shortHalfAxisB', (96.1560870104, 98.1560870104)),
    ('E2.rotationXerror', (-0.5, 0.5)),
    ('E2.rotationYerror', (-7.5, 7.5)),
    ('E2.rotationZerror', (22.0, 32.0)),
    ('E2.translationYerror', (-1.0, 1.0)),
    ('E2.translationZerror', (-1.0, 1.0)),

    ('ImagePlane.translationXerror', (-1.0, 1.0)),
    ('ImagePlane.translationYerror', (-1.0, 1.0)),
    ('ImagePlane.translationZerror', (-33.0, 33.0)),
])

param_list = list(param_limit_dict.keys())

backend = RayBackendLocalRayX(seed=123, verbose=True)
#backend = RayBackendLocalRayUI(seed=123)

In [55]:
outputs_dir = os.path.join(file_root, 'outputs/')
engine = MinMaxRayEngine(rml_basefile=os.path.join(file_root, 'testing/rml/METRIX_U41_G1_H1_318eV_PS_MLearn_1.15.rml'),
                                exported_planes=["ImagePlane"],
                                ray_backend=backend,
                                num_workers=-1,
                                param_limit_dict=param_limit_dict,
                                param_list=param_list,
                                as_generator=False,
                                verbose=True,
                  manual_transform_plane='ImagePlane',
                  manual_transform_xyz=('ImagePlane.translationXerror','ImagePlane.translationYerror','ImagePlane.translationZerror'))

In [56]:
seed = 1000042
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
parameters=torch.rand(1, len(param_list))
parameters =torch.ones_like(parameters)*1.
output = engine.run(parameters)

EXEC: rayx -S -s 123 /dev/shm/ray-workdir/whyt6w920rwdq2p0/workfile.rml
Starting rayx (, release)
Processing: "/dev/shm/ray-workdir/whyt6w920rwdq2p0/workfile.rml"
Finished in 22ms. Exported rays to: "/dev/shm/ray-workdir/whyt6w920rwdq2p0/workfile.h5"
Done. Processed 1 RML file(s)
[whyt6w920rwdq2p0] success in 0.04s (rc=0)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


In [57]:
output[0]['ray_output']

{'ImagePlane': RayOutput(x_loc=tensor([ 2.3010e+02,  2.1208e+01,  1.3031e+02,  2.2932e+01,  4.8457e+02,
          6.7428e+01,  4.3010e+01, -7.0127e+00,  8.1351e+00,  4.1056e+00,
         -1.4364e+02,  2.1607e+02,  6.9714e+01,  1.0846e+00,  1.6941e+01,
          9.9051e+01,  8.9048e+02, -3.7725e+00,  1.6061e+01, -3.0871e+01,
         -4.8637e+02, -4.7713e+01,  2.4647e+01,  4.4009e+01,  2.2383e+01,
         -1.0957e+02, -7.0925e+01,  2.1080e+01, -4.7175e+01,  4.7897e+00,
         -1.0429e+02,  1.3918e+02, -2.3256e+02, -5.3345e+00, -1.4239e+01,
          1.0104e+02,  2.7351e+00, -9.9378e+01, -1.9150e+02,  2.6894e+01,
          1.8438e+01,  8.6977e+00, -2.9852e+01,  5.2192e+01, -4.1702e+02,
         -1.1137e+03, -1.4184e+01,  5.7156e+00, -5.9719e+01,  3.7406e+00,
         -4.3874e+01, -3.9732e+01, -2.1230e+01, -5.2613e+01, -9.7105e+03,
         -8.5094e+00, -5.9212e+00,  1.7863e+01, -9.9055e+00, -7.3611e-01,
          4.4823e+01,  1.1126e+01,  2.1457e+01, -2.9629e+01,  3.1522e+01,
        

In [24]:
from ray_tools.simulation.data_tools import EfficientRandomRayDatasetGenerator
from ray_tools.base.transform import *

def sampler_func():
    return torch.rand(len(param_list))  # [P]
            
parameter_sampler = EfficientRandomRayDatasetGenerator.build_parameter_sampler(
    sampler_func=sampler_func,
    idx_sub=["ImagePlane"],
    transform=XYHistogram(50, (-10,10), (-2,2)),
)

gen = EfficientRandomRayDatasetGenerator(
    engine=engine,
    parameter_sampler=parameter_sampler,
    h5_datadir=outputs_dir,
    param_list=param_list,
    exported_planes=["ImagePlane"],
    h5_basename="dataset2",
    h5_max_size=20,
)

gen.generate(h5_idx=0, batch_size=50)

TypeError: ray_output[ImagePlane] must be RayOutput, got <class 'dict'>